# Check memory used during inference

In [ ]:
model.eval()

with torch.no_grad():
    occupancy_pred = []
    for batch in tqdm.tqdm(test_loader, ncols=100):
        coordinates = batch[0].cuda()  
        predictions = model(coordinates)
        occ_pred = predictions.detach().clone().cpu().numpy().flatten()
        occupancy_pred.append(occ_pred)
occupancy_pred = np.concat(occupancy_pred, axis=0)
occupancy_pred = occupancy_pred.reshape(resolution, resolution, resolution)

memory_before = memory_usage()[0]
# 8. apply marching cubes by reshaping it and extract mesh from the predictions
voxel_size = (1/resolution,)*3
vertices, faces, normals, values = marching_cubes(occupancy_pred, level=0.5, spacing=voxel_size)
memory_after = memory_usage()[0]
print(f'{memory_after - memory_before:.2f}MB')
# save and visualize mesh
mesh = trimesh.Trimesh(vertices, faces) 

# Post-processing: work after extracting mesh

In [ ]:
# color meshes
mesh.visual.vertex_colors = trimesh.visual.random_color()
# normalize mesh
mesh.fix_normals()
# centralize mesh: in our case to (-0.5,-0.5,-0.5) 
mesh.vertices += [-0.5,-0.5,-0.5]

# mesh.vertices = mesh.vertices - mesh.center_mass --> FYI

# invert surface toward outside
mesh.invert()

# mirroring & rotating --> FYI
vertices[:, 0] = -vertices[:, 0] # mirror on the x-axis
rotation_matrix = np.array([[0, 0, 1], [0, 1, 0], [-1, 0, 0]])
vertices = vertices @ rotation_matrix.T
normals = normals @ rotation_matrix.T # rotate by 90° on the x-z plane

# Trimesh object and sample points from the surface

In [ ]:
# object on 3D coordinates
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection="3d")

mesh_plot = Poly3DCollection(vertices[faces], alpha=0.7, facecolor="cyan", edgecolor="k")
ax.add_collection3d(mesh_plot)
ax.set_xlabel("x-axis")
ax.set_ylabel("y-axis")
ax.set_zlabel("z-axis")
plt.title("reconstructed mesh")
plt.show()
plt.savefig("reconstructed.png")

In [ ]:
# point cloud sampling
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
num_sample = 20000
point_cloud=mesh.sample(num_sample)

output_pointcloud_path = "sampled_point_cloud.npy"
np.save(output_pointcloud_path,point_cloud)
print(f"Point cloud saved.")

# Check if vertices are inside the expected range

In [ ]:
# 8. Apply Marching Cubes algorithm by reshaping it and extract mesh from the predictions
voxel_size = (1/resolution,)*3
vertices, faces, normals, values = marching_cubes(occupancy_pred, level=0.5, spacing=voxel_size)

# check if inside expected range [-0.5, 0.5]
min_vals = np.min(vertices, axis=0)
max_vals = np.max(vertices, axis=0)
print(f"Vertex coordinate min values: {min_vals}")
print(f"Vertex coordinate max values: {max_vals}")
